### 1. 한국어 금융 뉴스 긍정, 부정 분류
* https://github.com/ukairia777/finance_sentiment_corpus
* prediction('ChatGPT의 등장으로 인공지능 스타트업들이 비상이 걸렸다') ['부정']

### 2. 다중 클래스 분류
* https://github.com/kakaobrain/kor-nlu-datasets
* 두 문장을 입력받아 두 문장의 '얽힘', '중립', '모순'을 분류해주는 문제

* sent1 = '흡연자분들은 발코니가 있는 방이면 발코니에서 흡연하세요'
* sent2 = '이 건물은 흡연이 금지됩니다'
* prediction(sent1, sent2) ['모순']

### 3. 한국어 혐오 발언 다중 레이블 분류
* https://github.com/adlnlp/K-MHaS


In [1]:
!pip install datasets

In [42]:
import urllib.request
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datasets import Dataset, load_metric
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AutoTokenizer, pipeline

In [3]:
Dataset_url = 'https://github.com/ukairia777/finance_sentiment_corpus/blob/f1f76194c1ec8b136bc848272e2c0808444c8522/finance_data.csv?raw=true'
Dataset_name = 'finance_data.csv'
# 데이터 url 끝에 ?raw=true 붙여주면 다운 받아 직접 데이터를 확인할 수 있다.
# 눈으로 확인 한 결과: 한국어 인코딩 문제로 데이터를 불러오지 못함.

In [4]:
urllib.request.urlretrieve(Dataset_url, filename = Dataset_name)

('finance_data.csv', <http.client.HTTPMessage at 0x7bab03ad6980>)

In [5]:
df = pd.read_csv(Dataset_name, encoding='utf-8')
df.head()

,labels,sentence,kor_sentence
0,neutral,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [6]:
# 'sentence'컬럼은 영어 문장으로 필요 없어 제거.
df = df.drop(columns=['sentence'])
df.head()

,labels,kor_sentence
0,neutral,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [7]:
# 'labels' 컬럼의 값을 숫자로 변환
label_mapping = {'neutral': 0, 'positive': 1, 'negative': 2}
df['labels'] = df['labels'].map(label_mapping)
df.head()

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [8]:
df.isna().mean()

labels          0.0
kor_sentence    0.0
dtype: float64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   labels        4846 non-null   int64 
 1   kor_sentence  4846 non-null   object
dtypes: int64(1), object(1)
memory usage: 75.8+ KB


In [10]:
# 중복데이터 찾기
df[df.duplicated(subset=['kor_sentence'], keep=False)] # keep=False는 모든 중복된 행을 표시

,labels,kor_sentence
78,0,텔레콤월드와이어-2006년 4월 7일-TJ 그룹은 모닝 디지털 디자인 Oy 핀란드 ...
79,1,텔레콤월드와이어-2006년 4월 7일-TJ 그룹은 모닝 디지털 디자인 Oy 핀란드 ...
788,1,그룹의 사업은 스포츠의 광범위한 포트폴리오와 모든 주요 시장에서의 입지에 의해 균형...
789,0,그룹의 사업은 스포츠의 광범위한 포트폴리오와 모든 주요 시장에서의 입지에 의해 균형...
1098,0,이 발표 내용에 대한 책임은 전적으로 발행자에게 있습니다.
1099,0,이 발표 내용에 대한 책임은 전적으로 발행자에게 있습니다.
1393,0,"핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서..."
1394,0,"핀란드 헬싱키에 본사를 둔 레민카이넨 그룹은 토목 공학, 건축 계약, 기술 건축 서..."
1415,0,"이 보고서는 블랙 앤 데커, 피스카스, 피스카스 브랜드, 후스크바르나 아웃도어 프로..."
1416,0,"이 보고서는 블랙 앤 데커, 피스카스, 피스카스 브랜드, 후스크바르나 아웃도어 프로..."


In [11]:
# 데이터 중복 제거
df = df.drop_duplicates(subset=['kor_sentence'])
df.head()

,labels,kor_sentence
0,0,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."


In [12]:
# 중복된 데이터가 잘 삭제된 것을 볼 수 있다.
df[df.duplicated(subset=['kor_sentence'], keep=False)]

,labels,kor_sentence


In [13]:
# X_data 및 y_data 생성
X_data = df['kor_sentence']
y_data = df['labels']

In [14]:
# 데이터를 훈련 세트와 테스트 세트로 분할
# stratify=y_data는 y_data의 클래스 비율을 데이터 분할 시에도 비율을 유지해준다.
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=999, stratify=y_data)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

3861
966
3861
966


In [15]:
# y_train의 label 비율
y_train.value_counts(normalize=True)

labels
0    0.592852
1    0.282051
2    0.125097
Name: proportion, dtype: float64

In [16]:
y_test.value_counts(normalize=True)

labels
0    0.592133
1    0.282609
2    0.125259
Name: proportion, dtype: float64

In [17]:
# model 설정 / Hugging Face의 Transformers 라이브러리 중 BERT기반 모델 인 klue/roberta-base를 사용.
# klue/roberta-base는 한국어 RoBERTa 모델. KLUE 데이터셋을 기반으로 학습됨.
model_checkpoint = 'klue/roberta-base'
# 토그나이저 로드 / 텍스트를 모델이 이해할 수 있는 형식으로 변환.
# AutoTokenizer는 Transformers 라이브러리의 자동 토크나이저 클래스. 자동 선택.
# use_fast: Rust로 구현되 토크나이저로 일반적으로 더 빠른 성능을 제공.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
# 데이터셋 인코딩 / 텍스트 데이터를 모델 입력으로 사용할 수 있는 형식으로 변환
# truncation=True: 텍스트가 지정된 최대 길이(max_length)를 초과할 경우, 잘라냄. / 입력 크기를 일정하게 유지하여 메모리 사용을 최적화하고 모델 학습 속도를 높일 수 있음.
# padding=True: 텍스트가 지정된 최대 길이보다 짧을 경우, 부족한 부분을 패딩하여 채움.
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=128)

In [19]:
# 데이터셋 생성 / transformers 라이브러리의 Trainer와 호환되도록 데이터를 준비하는 중요한 단계
# Dataset.from_dict: 딕셔너리로부터 Dataset 객체를 생성. / train_encodings와 y_train 데이터를 딕셔너리로 구성
# input_ids: 토크나이저가 변환한 입력 텍스트의 토큰 ID 리스트. 각 단어 또는 서브워드는 고유의 숫자 ID로 변환.
# attention_mask: 입력 텍스트의 실제 단어와 패딩 토큰을 구분하기 위한 마스크. 실제 단어는 1, 패딩 토큰은 0으로 표시.
# labels: 각 입력 텍스트에 대한 레이블. 감정 분류 레이블(모순, 긍정, 부정)이 숫자로 변환되어 있음.
# .tolist() 메서드: Pandas Series를 리스트로 변환.
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'],
                                   'attention_mask': train_encodings['attention_mask'],
                                   'labels': y_train.tolist()})
test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'],
                                  'attention_mask': test_encodings['attention_mask'],
                                  'labels': y_test.tolist()})

In [20]:
# 데이터 collator
# transformers 라이브러리의 DataCollatorWithPadding 클래스는 배치(batch) 내의 모든 입력 텍스트가 동일한 길이가 되도록 패딩을 추가. 모델이 일관된 입력 크기로 작업할 수 있게 함.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [21]:
# 메트릭 로드
# F1 스코어: 정밀도(Precision)와 재현율(Recall)의 조화 평균으로, 특히 불균형 데이터에서 성능을 평가할 때 유용한 지표.
metric = load_metric('f1')

# eval_pred: Trainer 객체에서 제공하는 평가 예측 결과. 튜플은 모델의 로짓(logits)과 실제 레이블(labels)을 포함.
def compute_metrics(eval_pred):
    logits, labels = eval_pred # 로짓과 레이블로 분리
    predictions = np.argmax(logits, axis=-1) # 로짓에서 가장 높은 값을 가지는 인덱스를 예측값으로 사용
    return metric.compute(predictions=predictions, references=labels, average='macro') # F1 스코어를 계산하여 반환

<ipython-input-21-f041452b0a75>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('f1')


In [22]:
# 모델 로드
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
!pip install accelerate -U
!pip install transformers[torch]

In [24]:
# TrainingArguments 및 Trainer 설정
# 모델 학습 시 한 번에 처리할 데이터 샘플의 수를 지정.
batch_size = 64
# 모델 학습 시 필요한 설정을 지정하여 Trainer 객체에 전달하기 위해 작성.
training_args = TrainingArguments(
    output_dir='./results', # 모델, 출력을 저장할 디렉토리 지정.
    evaluation_strategy='epoch', # 매 에포크가 끝날 때마다 모델을 평가.
    save_strategy='epoch', # 매 에포크가 끝날 때마다 모델을 저장.
    learning_rate=2e-5, # 학습률을 설정 / 모델의 가중치를 업데이트할 때 사용하는 스텝 크기를 결정.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01, # 가중치 감소율을 설정 / 과적합을 방지하기 위해 가중치를 줄이는 기법.
    load_best_model_at_end=True, # 학습 종료 시 가장 좋은 성능을 보인 모델을 불러올지 여부를 설정.
    metric_for_best_model='f1', # F1 스코어를 사용하여 최적의 모델을 결정.
    logging_dir='./logs',
    logging_steps=50 # 지정된 스텝마다 로그를 기록
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    model=model,
    args=training_args, # 다양한 하이퍼파라미터 및 설정을 지정.
    train_dataset=train_dataset, # 학습에 사용할 데이터셋을 지정.
    eval_dataset=test_dataset, # 평가에 사용할 데이터셋을 지정.
    data_collator=data_collator, # 배치 데이터의 패딩 및 정렬을 처리하는 collator를 지정.
    tokenizer=tokenizer,
    compute_metrics=compute_metrics # 평가 예측 결과를 받아 로짓을 예측값으로 변환한 후, F1 스코어를 계산하여 반환.
)

In [26]:
# 모델 학습
# trainer.train()

In [27]:
# 모델 평가
# trainer.evaluate()

In [28]:
best_model_checkpoint = trainer.state.best_model_checkpoint
best_model_checkpoint

In [ ]:
# 최적의 모델 체크포인트 로드
model = AutoModelForSequenceClassification.from_pretrained(best_model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(best_model_checkpoint)

In [35]:
# model save
# torch.save(model.state_dict(), "BERT_model.pt")

In [38]:
# load the saved model (저장한 모델 불러오기)
model_path = '/content/drive/MyDrive/KDT/6. 자연어 처리/BERT_model.pt'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [39]:
# 텍스트 분류 작업을 수행할 파이프라인을 생성 / pipeline 함수는 다양한 NLP 작업을 위한 사전 정의된 파이프라인을 제공.
classifier = pipeline(
    'text-classification',
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True # 모든 클래스에 대한 예측 점수를 반환할지 여부를 설정.
)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [43]:
# 정확도 및 분류 보고서 출력
predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(-1)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['neutral', 'positive', 'negative']))

Accuracy: 0.8519668737060041
Classification Report:
              precision    recall  f1-score   support

     neutral       0.89      0.88      0.89       572
    positive       0.80      0.78      0.79       273
    negative       0.81      0.85      0.83       121

    accuracy                           0.85       966
   macro avg       0.83      0.84      0.84       966
weighted avg       0.85      0.85      0.85       966



In [54]:
# 레이블 이름 정의
label_names = ['중립', '긍정', '부정']

In [79]:
# 분류 예제
# example_text = 'IBK투자증권, 1천억원 규모 신종자본증권 발행 성공'
# example_text = '임종룡 금융 사고 뼈아파… 무신불립 신념으로 내부통제 강화'
# example_text = '총알 탄 트럼프 대세론, 비트코인 3% 뛰었다'
# example_text = '서국동 NH농협손보 대표, 호우피해 현장 점검 나서'
# example_text = '가상자산법 D-4… 이복현 금감원장 검찰과 불공정거래 엄정 대응'
example_text = '불 붙은 트래블 카드 경쟁..은행 본부 직원 동원에 노조 반발'
predictions = classifier(example_text)

max_label = max(predictions[0], key=lambda x: x['score'])
label = label_names[int(max_label['label'].split('_')[1])]

print(f"'{example_text}': ['{label}'] 입니다.")

'불 붙은 트래블 카드 경쟁..은행 본부 직원 동원에 노조 반발': ['부정'] 입니다.


### Micro 평균
* 계산 방법: 모든 클래스의 TP, FP, FN을 합산 후 메트릭 계산.
* 특징: 전체적인 분류 성능 측정에 적합.
* 적용 사례: 클래스 불균형이 심하고, 자주 발생하는 클래스의 성능이 중요할 때.
  * 전체적인 분류 성능 평가에 적합.

### Macro 평균
* 계산 방법: 각 클래스의 메트릭(F1 스코어)을 개별적으로 계산한 후 평균을 구함.
* 특징: 모든 클래스에 동일한 가중치를 부여.
* 적용 사례: 모든 클래스의 성능이 균등하게 중요할 때, 소수 클래스의 성능을 무시하지 않기 위해.
  * 각 클래스의 성능을 개별적으로 평가하는 데 유리.

### Weighted 평균
* 계산 방법: 각 클래스의 메트릭을 계산한 후, 각 클래스의 샘플 수에 따라 가중 평균을 구함.
* 특징: 클래스의 샘플 수에 따라 가중치를 부여.
* 적용 사례: 클래스 불균형이 존재하며, 각 클래스의 크기에 비례하여 성능을 측정하고 싶을 때.
  * 클래스 불균형이 있는 경우 각 클래스의 크기에 비례하여 성능을 평가하는 데 유리.